# Forward Kinematics Analysis

In [6]:
import numpy as np
from numpy import array
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2
from sympy.matrices import Matrix

In [7]:
#Create symbols for joint variables
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

In [8]:
###DH Parameter Table
#I need to set up the DH parameter table from the Kuka arm after setting 
#reference frames. 
#The values for a's and d's were found from the urdf xacro file.

s = {alpha0:     0, a0:      0, d1:  0.75,
     alpha1: -pi/2, a1:   0.35, d2:     0,   q2: q2-pi/2,
     alpha2:     0, a2:   1.25, d3:     0,
     alpha3: -pi/2, a3: -0.054, d4:  1.50,
     alpha4:  pi/2, a4:      0, d5:     0,
     alpha5: -pi/2, a5:      0, d6:     0,
     alpha6:     0, a6:      0, d7: 0.303,   q7: 0
    }

In [9]:
### Homogeneous Transforms
#These are found from base_link to link_7 which is the gripper

T0_1 = Matrix([[            cos(q1),            -sin(q1),            0,              a0],
               [sin(q1)*cos(alpha0), cos(q1)*cos(alpha0), -sin(alpha0), -sin(alpha0)*d1],
               [sin(q1)*sin(alpha0), cos(q1)*sin(alpha0),  cos(alpha0),  cos(alpha0)*d1],
               [                  0,                   0,            0,               1]])
T0_1 = T0_1.subs(s)

T1_2 = Matrix([[            cos(q2),            -sin(q2),            0,              a1],
               [sin(q2)*cos(alpha1), cos(q2)*cos(alpha1), -sin(alpha1), -sin(alpha1)*d2],
               [sin(q2)*sin(alpha1), cos(q2)*sin(alpha1),  cos(alpha1),  cos(alpha1)*d2],
               [                  0,                   0,            0,               1]])
T1_2 = T1_2.subs(s)

T2_3 = Matrix([[            cos(q3),            -sin(q3),            0,              a2],
               [sin(q3)*cos(alpha2), cos(q3)*cos(alpha2), -sin(alpha2), -sin(alpha2)*d3],
               [sin(q3)*sin(alpha2), cos(q3)*sin(alpha2),  cos(alpha2),  cos(alpha2)*d3],
               [                  0,                   0,            0,               1]])
T2_3 = T2_3.subs(s)

T3_4 = Matrix([[            cos(q4),            -sin(q4),            0,              a3],
               [sin(q4)*cos(alpha3), cos(q4)*cos(alpha3), -sin(alpha3), -sin(alpha3)*d4],
               [sin(q4)*sin(alpha3), cos(q4)*sin(alpha3),  cos(alpha3),  cos(alpha3)*d4],
               [                  0,                   0,            0,               1]])
T3_4 = T3_4.subs(s)

T4_5 = Matrix([[            cos(q5),            -sin(q5),            0,              a4],
               [sin(q5)*cos(alpha4), cos(q5)*cos(alpha4), -sin(alpha4), -sin(alpha4)*d5],
               [sin(q5)*sin(alpha4), cos(q5)*sin(alpha4),  cos(alpha4),  cos(alpha4)*d5],
               [                  0,                   0,            0,              1]])
T4_5 = T4_5.subs(s)

T5_6 = Matrix([[            cos(q6),            -sin(q6),            0,              a5],
               [sin(q6)*cos(alpha5), cos(q6)*cos(alpha5), -sin(alpha5), -sin(alpha5)*d6],
               [sin(q6)*sin(alpha5), cos(q6)*sin(alpha5),  cos(alpha5),  cos(alpha5)*d6],
               [                  0,                   0,            0,              1]])
T5_6 = T5_6.subs(s)

T6_G = Matrix([[            cos(q7),            -sin(q7),            0,              a6],
               [sin(q7)*cos(alpha6), cos(q7)*cos(alpha6), -sin(alpha6), -sin(alpha6)*d7],
               [sin(q7)*sin(alpha6), cos(q7)*sin(alpha6),  cos(alpha6),  cos(alpha6)*d7],
               [                  0,                   0,            0,              1]])
T6_G = T6_G.subs(s)


In [10]:
#Composition of Homogeneous Transforms from base to gripper
T0_2 = simplify(T0_1 * T1_2) # base_link to link_2
T0_3 = simplify(T0_2 * T2_3) # base_link to link_3
T0_4 = simplify(T0_3 * T3_4) # base_link to link_4
T0_5 = simplify(T0_4 * T4_5) # base_link to link_5
T0_6 = simplify(T0_5 * T5_6) # base_link to link_6
T0_G = simplify(T0_6 * T6_G) # base_link to link_G(which is the gripper)

print('Homogeneous Transform from base_link to gripper_link is \n')
for i in T0_G:
    print(i)
    print('')


Homogeneous Transform from base_link to gripper_link is 

((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*cos(q6) + (sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1))*sin(q6)

-((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*sin(q6) + (sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1))*cos(q6)

-(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) + cos(q1)*cos(q5)*cos(q2 + q3)

-0.303*(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) + (1.25*sin(q2) - 0.054*sin(q2 + q3) + 1.5*cos(q2 + q3) + 0.35)*cos(q1) + 0.303*cos(q1)*cos(q5)*cos(q2 + q3)

((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*cos(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*sin(q6)

-((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*sin(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*cos(q6)

-(sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos

In [11]:
#There is a correction needed due to the difference in the definition of the Gripper
#link in URDF versus DH Convention. We start with building a rotation about the 
#z axis and the y axis
R_z = Matrix([[cos(np.pi), -sin(np.pi), 0, 0],
              [sin(np.pi),  cos(np.pi), 0, 0],
              [         0,           0, 1, 0],
              [         0,           0, 0, 1]])
R_y = Matrix([[ cos(-np.pi/2), 0, sin(-np.pi/2), 0],
              [             0, 1,             0, 0],
              [-sin(-np.pi/2), 0, cos(-np.pi/2), 0],
              [             0, 0,             0, 1]])
R_corr = simplify(R_z * R_y)


In [12]:
### Numerically evaluate transforms in order to compare to the tf_echo
## and check my equations
#print("T0_1 = ",T0_1.evalf(subs={q1: 0.17, q2: 0.03, q3: 0.22, q4: -5.28, q5: -1.97, q6: 5.39}))
#print("T0_2 = ",T0_2.evalf(subs={q1: 0.17, q2: 0.03, q3: 0.22, q4: -5.28, q5: -1.97, q6: 5.39}))
#print("T0_3 = ",T0_3.evalf(subs={q1: 0.17, q2: 0.03, q3: 0.22, q4: -5.28, q5: -1.97, q6: 5.39}))
#print("T0_4 = ",T0_4.evalf(subs={q1: 0.17, q2: 0.03, q3: 0.22, q4: -5.28, q5: -1.97, q6: 5.39}))
#print("T0_5 = ",T0_5.evalf(subs={q1: 0.17, q2: 0.03, q3: 0.22, q4: -5.28, q5: -1.97, q6: 5.39}))
#print("T0_6 = ",T0_6.evalf(subs={q1: 0.17, q2: 0.03, q3: 0.22, q4: -5.28, q5: -1.97, q6: 5.39}))
#print("T0_G = ",T0_G.evalf(subs={q1: 0.17, q2: 0.03, q3: 0.22, q4: -5.28, q5: -1.97, q6: 5.39}))

#Total Homogeneous Transform between the Base_link and the Gripper_link with
#the orientation correction applied
T_total = simplify(T0_G * R_corr)
#print("T_total = ",T_total.evalf(subs={q1: 0.17, q2: 0.03, q3: 0.22, q4: -5.28, q5: -1.97, q6: 5.39}))



## Inverse Kinematics Analysis

In [35]:
## We first need to solve for the cartesian coordinates of the Wrist Center
#We will use the following formula's
# w_x = p_x -(d6 + l) * n_x, w_y = p_y -(d6 + l) * n_y, w_y = p_y -(d6 + l) * n_y
#n_x, n_y, and n_z is the third column in the homogeneous tranform from base_link to gripper_link
# The total rotation matrix is shown below
#roll, pitch, yaw are from the orientation message from the gripper
R_T = Matrix([[cos(roll)*cos(pitch), cos(roll)*sin(pitch)*sin(yaw) - sin(roll)*cos(yaw), 
                cos(roll)*sin(pitch)*cos(yaw) + sin(roll)*sin(yaw)],
              [sin(roll)*cos(pitch), sin(roll)*sin(pitch)*sin(yaw) + cos(roll)*cos(yaw), 
                 sin(roll)*sin(pitch)*cos(yaw) - cos(roll)*sin(yaw)],
              [-sin(pitch), cos(pitch)*sin(yaw), cos(pitch)*cos(yaw)]])
#Nx, Ny, and Nz is the third column in the homogeneous tranform from base_link to gripper_link or
#the third column from the total composite rotation matrix
Nx = R_T[0,2]
Ny = R_T[1,2]
Nz = R_T[2,2]

#px, py, and pz is given from the end effector pose

#d6 is from the DH parameter table
d6 = 0

#l is the length of the end-effector which is the link offset d7 from the DH table
l = 0.303

#Now we can calculate the wrist center position
WCx = px - (d6 + l)*Nx
WCy = py - (d6 + l)*Ny
WCz = pz - (d6 + l)*Nz

#We can evaluate the wrist center position when given joint variables
#print("w_x = ",w_x.evalf(subs={q1: 1.83, q2: -0.73, q3: -2.42, q4: 3.75, q5: 1.86, q6: 2.43}))
#print("w_y = ",w_y.evalf(subs={q1: 1.83, q2: -0.73, q3: -2.42, q4: 3.75, q5: 1.86, q6: 2.43}))
#print("w_z = ",w_z.evalf(subs={q1: 1.83, q2: -0.73, q3: -2.42, q4: 3.75, q5: 1.86, q6: 2.43}))

#We can set up the joint variables 1 to 3 equations
#a is equal to a2 from the DH parameter table
a = 1.25
#b was found using trig
b = sqrt(w_y**2 + a**2 - 2*(a*w_x) + w_x**2)
#D was found using the law of cosines
D = (w_x**2 + w_y**2 - a**2 - b**2)/(2*a*b)
#Here are the formulas for the first three joint variables
q_1 = atan2(w_y, w_x)
q_2 = atan2(D, sqrt(1 - D**2))
q_3 = atan2(b, a)


w_x =  0.895897318313310
w_y =  -1.95157132383425
w_z =  1.54283130070845
-1.14042989329561
-0.179492148192444
1.00845253664840


In [22]:
#This section was testing code before put into the inverse kinematics python file
# The total rotation matrix is shown below
roll, pitch, yaw = 0, 0, 0
R_T = Matrix([[cos(roll)*cos(pitch), cos(roll)*sin(pitch)*sin(yaw) - sin(roll)*cos(yaw), 
                cos(roll)*sin(pitch)*cos(yaw) + sin(roll)*sin(yaw)],
              [sin(roll)*cos(pitch), sin(roll)*sin(pitch)*sin(yaw) + cos(roll)*cos(yaw), 
                 sin(roll)*sin(pitch)*cos(yaw) - cos(roll)*sin(yaw)],
              [-sin(pitch), cos(pitch)*sin(yaw), cos(pitch)*cos(yaw)]])
Nx = R_T[0,2]
Ny = R_T[1,2]
Nz = R_T[2,2]
q1 = 0.17
q2 = 0.03
q3 = 0.22
T0_3 = T0_3.evalf(subs={q1: q1, q2: q2, q3: q3})
R0_3 = T0_3[0:3,0:3]
T3_5 = T3_4 * T4_5
T3_6 = T3_5 * T5_6
R3_6 = T3_6[0:3,0:3]
print(R3_6)
print(R3_6[2,2])
print(Nx)
print(Ny)
print(Nz)


Matrix([[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4)], [sin(q5)*cos(q6), -sin(q5)*sin(q6), cos(q5)], [-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4), sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6), sin(q4)*sin(q5)]])
sin(q4)*sin(q5)
0
0
1
